# Working with Message Stack payloads 
- there are currently three support schemes; openai, google, anthropic
- these have slightyl different payload structure for tool calls and responses in particular
- generally, we want to ACK a tool call with an id and follow it with a response in the message stack
- anthropic has a tool block and google has a functionResponse while open AI is easier with just the typically message with role and content
- we can test read message stacks as instructions from the database in different contexts
    - for a user question; trivial 
    - for a tool request with tool stack
    - for agents that provider system prompts

In [ ]:
#open ai example
{
  "model": "gpt-4-turbo",
  "messages": [
    {
      "role": "user",
      "content": "Give me election voting info and check the latest news on AI."
    },
    {
      "role": "assistant",
      "content": null,
      "tool_calls": [
        {
          "id": "tool_1",
          "type": "function",
          "function": {
            "name": "get_policy",
            "arguments": {
              "category": "election_voting"
            }
          }
        },
        {
          "id": "tool_2",
          "type": "function",
          "function": {
            "name": "search",
            "arguments": {
              "query": "latest news on AI"
            }
          }
        }
      ]
    },
    {
      "role": "tool",
      "tool_use_id": "tool_1",
      "name": "get_policy",
      "content": "Election voting information: [Details here...]"
    },
    {
      "role": "tool",
      "tool_use_id": "tool_2",
      "name": "search",
      "content": "Latest AI news: [Top headlines here...]"
    }
  ]
}

In [225]:
def get_message_stack(question:str, session_id: str=None, agent_or_system_prompt:str=None, scheme:str='openai' ):
    """
    the stack is built with just the questions and prompt unless there are data in the session
    the canonical form has roles; user, tool, assistant
    for google the role maps not user to model
    for anthropic the role maps tool to user
    
    for google the parts are mapped
    
    """
    pass

In [226]:
import sys
sys.path.append('../')
import percolate as p8
from percolate.models.p8 import AIResponse
fns =[{
  "name": "get_weather",
  "description": "Get the weather forecast for a specific city and date",
  "parameters": {
    "type": "object",
    "properties": {
      "city": {
        "type": "string",
        "description": "The city for which to get the weather forecast"
      },
      "date": {
        "type": "string",
        "description": "The date for the weather forecast (YYYY-MM-DD)"
      }
    },
    "required": ["city", "date"]
  }
}]
#p8.repository(AIResponse).register()

In [227]:

from percolate.services import PostgresService
import uuid
uuid.uuid1()

In [228]:

from pydantic import BaseModel, model_validator, Field
import typing

class OpenAIMessage(BaseModel):
    role: str
    content: typing.Optional[str] = Field('', desscription='text content')
    tool_calls: typing.Optional[typing.List[dict]|dict]
    tool_call_id: typing.Optional[str] = None
    @model_validator(mode='before')
    @classmethod
    def _val(cls,values):
        if tool_calls:= values.get('tool_calls'):
            if isinstance(tool_calls,dict):
                values['tool_calls'] = [tool_calls]
        return values

pg = PostgresService()


sid='d0d4a69c-dd9d-11ef-b115-7606330c2360'
q = AIResponse( id=str(uuid.uuid1()), session_id=sid, role='user', content="What's the weather in Paris tomorrow?", tokens_in=0, tokens_out=0, status='QUESTION', model_name='gpt-4o-mini')
r = AIResponse.from_open_ai_response({'id': 'chatcmpl-AugmVtbKSogrlrGVbaENikjA5JCvl',
 'object': 'chat.completion',
 'created': 1738074183,
 'model': 'gpt-4o-mini-2024-07-18',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': None,
    'tool_calls': [{'id': 'call_YCzly4yPh2l1QDP0WYZVi1K8',
      'type': 'function',
      'function': {'name': 'get_weather',
       'arguments': '{"city":"Paris","date":"2023-10-05"}'}}],
    'refusal': None},
   'logprobs': None,
   'finish_reason': 'tool_calls'}],
 'usage': {'prompt_tokens': 81,
  'completion_tokens': 24,
  'total_tokens': 105,
  'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0},
  'completion_tokens_details': {'reasoning_tokens': 0,
   'audio_tokens': 0,
   'accepted_prediction_tokens': 0,
   'rejected_prediction_tokens': 0}},
 'service_tier': 'default',
 'system_fingerprint': 'fp_72ed7ab54c'},sid=sid)

a = AIResponse( id=str(uuid.uuid1()), session_id=sid, role='tool', content="Its pretty nice", tool_eval_data={'id': 'call_YCzly4yPh2l1QDP0WYZVi1K8','content':"Its pretty nice", 'name': 'get_weather'}, tokens_in=0, tokens_out=0, status='TOOL_RESPONSE', model_name='gpt-4o-mini')
pg.update_records([r,a])


In [229]:
from percolate.models.p8.types import _OpenAIMessage

mm = [d for d in pg.execute(f"""  select * from p8.get_anthropic_messages('what is the waether in paris on 2023-10-05', 'd0d4a69c-dd9d-11ef-b115-7606330c2360', 'use the functions to answer the users question') """)[0]['messages']]
mm

In [239]:
mm = [d for d in pg.execute(f""" select * from p8.get_canonical_messages('b5c70717-a5a1-390e-bceb-1c5b064016a9') """)[0]['messages']]
mm

In [242]:
data.json()

In [241]:
data = request_openai(mm)
data

In [216]:
def combine_system_text(data):
    system_text = '\n'.join(
        item['content'][0]['text'] for item in data if item['role'] == 'system'
    )
    return system_text
combine_system_text(mm)

In [219]:
## Open AI Scheme

import requests
import json
import os

def request_openai(messages,functions=fns):
    """

    """
    #mm = [_OpenAIMessage.from_message(d) for d in pg.execute(f"""  select * from p8.get_canonical_messages(NULL, '2bc7f694-dd85-11ef-9aff-7606330c2360') """)[0]['messages']]
    #request_openai(mm)
    
    """place all system prompts at the start"""
    
    messages = [m if isinstance(m,dict) else m.model_dump() for m in messages]
    url = "https://api.openai.com/v1/chat/completions"
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {os.environ.get('OPENAI_API_KEY')}"
    }

    data = {
        "model": "gpt-4o-mini",
        "messages": messages,
        "tools":  [{'type': 'function', 'function': f} for f in fns or []]
    }
    
    return requests.post(url, headers=headers, data=json.dumps(data))
 

def request_google(messages, functinos=fns):
    """
    https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/function-calling
    
    expected tool call parts [{'functionCall': {'name': 'get_weather', 'args': {'date': '2024-07-27', 'city': 'Paris'}}}]
    """
    def last_message_not_function_response(items):
        if not items:
            return True
        msg = items[-1]
        print(msg)
        if 'functionResponse' in msg['parts'][0].keys():
            return False
        
        return True
        
        
    system_prompt = [m for m in messages if m['role']=='system']

    url = f"https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash:generateContent?key={os.environ.get('GEMINI_API_KEY')}"
    headers = {
        "Content-Type": "application/json"
    }
    
    """important not to include system prompt - you can get some cryptic messages"""
    data = {
        "contents": [m for m in messages if m['role'] !='system']
    }
     
    if system_prompt:
        data['system_instruction'] = '\n'.join( item['content'][0]['text'] for item in system_prompts )
    
    """i have seen gemini call the tool even when it was the data if this mode is set"""
    if fns and last_message_not_function_response(messages):
        data.update(
           { "tool_config": {
              "function_calling_config": {"mode": "ANY"}
            },
            "tools": [{'function_declarations': fns}]}
        )
        
    return requests.post(url, headers=headers, data=json.dumps(data))
 

def request_anthropic(messages, functinos=fns):
    url = "https://api.anthropic.com/v1/messages"
    headers = {
        "Content-Type": "application/json",
        "x-api-key":  os.environ.get('ANTHROPIC_API_KEY'),
        "anthropic-version": "2023-06-01",
    }
    
    
    def _adapt_tools_for_anthropic( functions: typing.List[dict]):
            """slightly different dialect of function wrapper - rename parameters to input_schema"""
            def _rewrite(d):
                return {
                    'name' : d['name'],
                    'input_schema': d['parameters'],
                    'description': d['description']
                } 
            return [_rewrite(f) for f in functions]


    data = {
        "model": "claude-3-5-sonnet-20241022",
        "max_tokens": 1024,
        "messages": [m for m in messages if m['role'] !='system'],
        "tools": _adapt_tools_for_anthropic(fns) if fns else None
    }
    
    system_prompt = [m for m in messages if m['role']=='system']
   
    if system_prompt:
        data['system'] = '\n'.join( item['content'][0]['text'] for item in system_prompt )
    
    return requests.post(url, headers=headers, data=json.dumps(data))


In [220]:
# url = "https://api.anthropic.com/v1/messages"
# headers = {
#     "Content-Type": "application/json",
#     "x-api-key":  os.environ.get('ANTHROPIC_API_KEY'),
#     "anthropic-version": "2023-06-01",
# }

# data = {
#     "model": "claude-3-5-sonnet-20241022",
#     "max_tokens": 1024,
#     "messages": [
#         {"role": "user", "content": "What's the weather in Paris tomorrow?"}
#     ],
#     "tools":_adapt_tools_for_anthropic(fns) if fns else None
# }
# data = requests.post(url, headers=headers, data=json.dumps(data))
# data.json()

In [222]:
request_anthropic(mm).json()